# A deep learning demo

This is a Jupyter notebook, live in your browser, with executable Python code!

In [ ]:
a = 10
b = 4*8
print(a+b)

If running from Google Colab:

In [ ]:
!git clone https://github.com/glouppe/blackbelt 

In [ ]:
%cd blackbelt

# Use a pre-trained neural network

In [ ]:
from keras.preprocessing.image import load_img

In [ ]:
# load an image from file
image = load_img("data/dogs-vs-cats/train/dogs/dog.1106.jpg", target_size=(224, 224))
#image = load_img("data/dogs-vs-cats/train/cats/cat.1553.jpg", target_size=(224, 224))

In [ ]:
image

In [ ]:
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

In [ ]:
# load the model
model = VGG16()

In [ ]:
# convert the image pixels to a numpy array
image = img_to_array(image)

# reshape data for the model
image = np.expand_dims(image, axis=0)

# prepare the image for the VGG model
image = preprocess_input(image)

# predict the probability across all output classes
yhat = model.predict(image)

# convert the probabilities to class labels
label = decode_predictions(yhat)

# retrieve the most likely result
label = label[0][0]

# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

It works!

# Train a small convolutional network from scratch

In [ ]:
# location of dogs and cats
train_data_dir = "data/dogs-vs-cats/train"
validation_data_dir = "data/dogs-vs-cats/validation"

In [ ]:
!ls -l "data/dogs-vs-cats/train"

In [ ]:
!ls -l "data/dogs-vs-cats/train/dogs" 

In [ ]:
# data iterators
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary")

validation_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary")

In [ ]:
# convolutional network
from keras.models import Model 
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

input_shape = (img_width, img_height, 3)

inputs = Input(shape=input_shape)
h = Conv2D(32, (3, 3), activation="relu")(inputs)
h = MaxPooling2D(pool_size=(2, 2))(h)
h = Conv2D(32, (3, 3), activation="relu")(h)
h = MaxPooling2D(pool_size=(2, 2))(h)
h = Conv2D(32, (3, 3), activation="relu")(h)
h = MaxPooling2D(pool_size=(2, 2))(h)
h = Flatten()(h)
h = Dense(64, activation="relu")(h)
h = Dropout(0.5)(h)
predictions = Dense(1, activation="sigmoid")(h)

model = Model(inputs=inputs, outputs=predictions)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
# train
nb_train_samples = 2000
nb_validation_samples = 2000
epochs = 1

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Final accuracy at 70%.

In [ ]:
# sample prediction 1
img_path = "data/dogs-vs-cats/validation/cats/cat.2798.jpg"
image = load_img(img_path, target_size=(img_width, img_height))
image

In [ ]:
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
model.predict(image)[0, 0]

In [ ]:
# sample prediction 2
img_path = "data/dogs-vs-cats/validation/dogs/dog.2774.jpg"
image = load_img(img_path, target_size=(img_width, img_height))
image

In [ ]:
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
model.predict(image)[0, 0]

# Fine-tune the top layer of a pre-trained network

In [ ]:
# location of dogs and cats
train_data_dir = "data/dogs-vs-cats/train"
validation_data_dir = "data/dogs-vs-cats/validation"

# data iterators
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = (224, 224)
batch_size = 32

def preprocess_input_vgg(x):
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_vgg,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary")

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)

validation_generator = validation_datagen.flow_from_directory(
    directory=validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary")

In [ ]:
# load vgg16 and fine-tune its top layer
from keras.applications import VGG16
from keras.layers import Dense
from keras.models import Model 

base_model = VGG16(weights="imagenet")
h = base_model.get_layer("fc2").output
predictions = Dense(1, activation="sigmoid", name="predictions")(h)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    if layer.name in ["predictions"]:
        continue
    layer.trainable = False
    
model.compile(optimizer="rmsprop", 
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
# train
nb_train_samples = 2000
nb_validation_samples = 2000
epochs = 1

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Final accuracy at 98%!

In [ ]:
# sample prediction 1
img_path = "data/dogs-vs-cats/validation/cats/cat.2010.jpg"
image = load_img(img_path, target_size=(img_width, img_height))
image

In [ ]:
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
model.predict(image)[0, 0]

In [ ]:
# sample prediction 2
img_path = "data/dogs-vs-cats/validation/dogs/dog.2012.jpg"
image = load_img(img_path, target_size=(img_width, img_height))
image

In [ ]:
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
model.predict(image)[0, 0]